In [93]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from torch import nn

In [94]:
csv_file_path = './part-00000-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv'
raw_data = pd.read_csv(csv_file_path)

In [95]:
# print(raw_data.head(5))

In [96]:
# iloc: integer location [row, column]
# store only data (not column name)
# use iloc, not just slicing.
data = raw_data.iloc[1:, :-1]
label = raw_data.iloc[:, -1]

In [97]:
# print(data.shape)
# print(data.head(5))
# print(label.shape)
# print(label.head(5))

In [98]:
unique_label = label.unique()

In [99]:
# print(len(unique_label))
# print(unique_label)

In [100]:
print(raw_data.columns)

Index(['flow_duration', 'Header_Length', 'Protocol Type', 'Duration', 'Rate',
       'Srate', 'Drate', 'fin_flag_number', 'syn_flag_number',
       'rst_flag_number', 'psh_flag_number', 'ack_flag_number',
       'ece_flag_number', 'cwr_flag_number', 'ack_count', 'syn_count',
       'fin_count', 'urg_count', 'rst_count', 'HTTP', 'HTTPS', 'DNS', 'Telnet',
       'SMTP', 'SSH', 'IRC', 'TCP', 'UDP', 'DHCP', 'ARP', 'ICMP', 'IPv', 'LLC',
       'Tot sum', 'Min', 'Max', 'AVG', 'Std', 'Tot size', 'IAT', 'Number',
       'Magnitue', 'Radius', 'Covariance', 'Variance', 'Weight', 'label'],
      dtype='object')


In [101]:
label_counts = raw_data['label'].value_counts()
print(label_counts)

label
DDoS-ICMP_Flood            36554
DDoS-UDP_Flood             27626
DDoS-TCP_Flood             23149
DDoS-PSHACK_Flood          21210
DDoS-SYN_Flood             20739
DDoS-RSTFINFlood           20669
DDoS-SynonymousIP_Flood    18189
DoS-UDP_Flood              16957
DoS-TCP_Flood              13630
DoS-SYN_Flood              10275
BenignTraffic               5600
Mirai-greeth_flood          5016
Mirai-udpplain              4661
Mirai-greip_flood           3758
DDoS-ICMP_Fragmentation     2377
MITM-ArpSpoofing            1614
DDoS-ACK_Fragmentation      1505
DDoS-UDP_Fragmentation      1484
DNS_Spoofing                 925
Recon-HostDiscovery          697
Recon-OSScan                 517
Recon-PortScan               430
DoS-HTTP_Flood               414
VulnerabilityScan            210
DDoS-HTTP_Flood              169
DDoS-SlowLoris               106
DictionaryBruteForce          63
SqlInjection                  31
BrowserHijacking              30
CommandInjection              28
Back

In [102]:
features = raw_data.drop('label', axis = 1)
target = raw_data['label']

# print(len(features))
# print(features)
# print(len(target))
# print(target)

# For label, do label encoding

In [103]:
target, target_mapping = pd.factorize(target)

In [89]:
print(target)
print(target_mapping)

[0 1 2 ... 8 9 2]
Index(['DDoS-RSTFINFlood', 'DoS-TCP_Flood', 'DDoS-ICMP_Flood', 'DoS-UDP_Flood',
       'DoS-SYN_Flood', 'Mirai-greeth_flood', 'DDoS-SynonymousIP_Flood',
       'Mirai-udpplain', 'DDoS-SYN_Flood', 'DDoS-PSHACK_Flood',
       'DDoS-TCP_Flood', 'DDoS-UDP_Flood', 'BenignTraffic', 'MITM-ArpSpoofing',
       'DDoS-ACK_Fragmentation', 'Mirai-greip_flood', 'DoS-HTTP_Flood',
       'DDoS-ICMP_Fragmentation', 'Recon-PortScan', 'DNS_Spoofing',
       'DDoS-UDP_Fragmentation', 'Recon-OSScan', 'XSS', 'DDoS-HTTP_Flood',
       'Recon-HostDiscovery', 'CommandInjection', 'VulnerabilityScan',
       'DDoS-SlowLoris', 'Backdoor_Malware', 'BrowserHijacking',
       'DictionaryBruteForce', 'SqlInjection', 'Recon-PingSweep',
       'Uploading_Attack'],
      dtype='object')


# Divide dataset into test and train data

In [126]:
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.2, random_state=42)

In [127]:
print(type(features_train))

# Since it's still pandas dataframe, translate into numpy array and then Tensor.
print(type(features_train.values))

<class 'pandas.core.frame.DataFrame'>
<class 'numpy.ndarray'>


In [128]:
features_train = torch.Tensor(features_train.values)
features_test = torch.Tensor(features_test.values)

target_train = torch.Tensor(target_train)
target_test = torch.Tensor(target_test)

In [129]:
train_dataset = TensorDataset(features_train, target_train)
test_dataset = TensorDataset(features_test, target_test)

# Set hyperparameters

In [92]:
batch_size = 64

In [130]:
train_dataloader = DataLoader(train_dataset, batch_size=batch_size) #shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

# Make custom dataset

In [59]:
# # X_tensor = torch.from_numpy(features.values)
# # X_tensor = torch.FloatTensor(features.values)
# # y_tensor = torch.from_numpy(target)

# class CustomDataset(Dataset):
#     def __init__(self, X, y):
#         # X is features, y is targets
#         self.X = X
#         self.y = y

#     def __len__(self):
#         return len(self.X)
    
#     def __getitem__(self, idx):
#         a = self.X[idx]
#         b = self.y[idx]
#         return a, b

# dataset = CustomDataset(X_tensor, y_tensor)

# set device

In [61]:
# setting up the device
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

print(f"Using {device} device")
# Using mps device

Using mps device


# define model

In [70]:
# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(46,46),
            nn.ReLU(),
            nn.Linear(46,34)
        )

    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device=device)

print(model)

NeuralNetwork(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=46, out_features=46, bias=True)
    (1): ReLU()
    (2): Linear(in_features=46, out_features=34, bias=True)
  )
)


# define loss fucntion and optimizer

In [71]:
# Optimizing the model parameters
# To train a model, we need a loss function and an optimizer.
lr = 0.001
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=lr)

# define train and test function

In [72]:
from torch.autograd import Variable
# In a single training loop, the model makes predictinos on the training dataset (fed to it in batches)
# and backpropagates the predictino error to adjust the model's parameters
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute predictino error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f} [{current:>5d}|{size:>5d}]")

# We also check the model's performance against the test dataset to ensure it is learning.
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
        test_loss /= num_batches
        correct /= size
        print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f}\n")

# train the model

In [73]:
# The training process is conducted over several iterations (epochs).
# Durin each epoch, the model learns parameters to make better predictions.
# Print the model's accuracy and loss at each epoch.
# see the accuracy increase and the loss cecrease with every epoch.

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n--------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
--------------------------------
loss: 3873877.500000 [   64|238687]
loss: 3.558402 [ 6464|238687]
loss: 3.532887 [12864|238687]
loss: 3.546443 [19264|238687]
loss: 3.528836 [25664|238687]
loss: 3.508924 [32064|238687]
loss: 3.524583 [38464|238687]
loss: 3.515392 [44864|238687]
loss: 3.506018 [51264|238687]
loss: 3.500764 [57664|238687]
loss: 3.499952 [64064|238687]
loss: 3.481689 [70464|238687]
loss: 3.481556 [76864|238687]
loss: 3.483852 [83264|238687]
loss: 3.457346 [89664|238687]
loss: 3.480426 [96064|238687]
loss: 3.444626 [102464|238687]
loss: 3.460358 [108864|238687]
loss: 3.453791 [115264|238687]
loss: 3.419555 [121664|238687]
loss: 3.442797 [128064|238687]
loss: 3.437433 [134464|238687]
loss: 3.416757 [140864|238687]
loss: 3.429369 [147264|238687]
loss: 3.421117 [153664|238687]
loss: 3.412008 [160064|238687]
loss: 3.412659 [166464|238687]
loss: 3.380772 [172864|238687]
loss: 3.413282 [179264|238687]
loss: 3.364743 [185664|238687]
loss: 3.396656 [192064|238687]
loss: 3.

KeyboardInterrupt: 